In [ ]:
import matplotlib.pyplot as plt
from Genetic.Representation import Function
from Genetic.NeuroEvolution import NeuroEvolution
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.losses import mse
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import seaborn as sns
from IPython.display import display
import numpy as np
from Model.model import SurvModel, FlchainModel, NwtcoModel, SupportModel
from Data.dataset import Flchain, Support, Nwtco
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import f_oneway, ttest_rel

from Utils.utils import Experiment, FunctionHelper, StatsHelper
from itertools import combinations

In [ ]:
surv_reg1_func = [1, 9, 9, 1, 5, 12, 7, 9, 7]
surv_reg2_func = [1, 9, 12, 1, 5, 12, 7, 9, 7]

surv_reg1_f = FunctionHelper(surv_reg1_func)
surv_reg2_f = FunctionHelper(surv_reg2_func)

surv_reg1_f.plot_surv_loss()
surv_reg2_f.plot_surv_loss()

In [ ]:
model_class = FlchainModel
ds = Flchain('Data/flchain.csv')

In [ ]:
(x_train, ye_train, y_train, e_train,
 x_val, ye_val, y_val, e_val,
 x_test, ye_test, y_test, e_test) = ds.get_train_val_test_from_splits(test_id=0, val_id=1)
xy = (x_train, ye_train), (x_val, ye_val), (x_test, ye_test)

In [ ]:
exp_fix_left = [
    [
    [6, 20, 23, 4, 5, 16, 13, 9, 17],
    [0.7745944, 0.79208297, 0.8049032, 0.80975556, 0.7876747]
    ]
    ,
    [
    [1, 29, 24, 5, 3, 14, 17, 26, 19],
    [0.7747819, 0.78683347, 0.8031153, 0.80985254, 0.7875028]
    ]
    ,
    [
    [4, 17, 17, 6, 6, 9, 14, 21, 23],
    [0.7753582, 0.79165417, 0.7987482, 0.8105683, 0.7863662]
    ]
    ,
    [
    [4, 24, 23, 2, 5, 8, 7, 10, 17],
    [0.7734723, 0.79130405, 0.8007376, 0.8101925, 0.788057]
    ]
    ,
    [
    [1, 22, 29, 3, 5, 20, 20, 25, 15],
    [0.7704736, 0.77819854, 0.7961018, 0.80588037, 0.7836693]
    ]
    ,
    [
    [2, 14, 19, 3, 5, 18, 22, 18, 21],
    [0.7725109, 0.78236294, 0.7989819, 0.79936737, 0.7840638]
    ]
    ,
    [
    [1, 8, 25, 4, 6, 17, 17, 27, 27],
    [0.7659427, 0.7819791, 0.8006382, 0.8021709, 0.7836167]
    ]
    ,
    [
    [2, 23, 9, 1, 5, 30, 9, 11, 17],
    [0.77448267, 0.7900953, 0.7970422, 0.80844843, 0.7835331]
    ]
    ,
    [
    [2, 10, 24, 1, 6, 20, 22, 9, 21],
    [0.7698465, 0.7841193, 0.79845744, 0.80669683, 0.7823861]
    ]
    ,
    [
    [4, 23, 24, 2, 2, 20, 21, 24, 8],
    [0.7737715, 0.79147536, 0.8014075, 0.8093111, 0.7872313]
    ]
]


exp_fix_right = [
    [
    [6, 10, 12, 3, 1, 16, 17, 24, 12],
    [0.7676365, 0.792303, 0.78690565, 0.79627377, 0.78267455]
    ]
    ,
    [
    [5, 24, 8, 1, 1, 10, 11, 23, 15],
    [0.7675293, 0.7878147, 0.790901, 0.7998203, 0.7842207]
    ]
    ,
    [
    [3, 17, 30, 2, 5, 23, 24, 30, 13],
    [0.77090764, 0.7898594, 0.78968084, 0.8010954, 0.78246975]
    ]
    ,
    [
    [2, 19, 8, 4, 2, 27, 18, 16, 29],
    [0.7764212, 0.7927412, 0.79930216, 0.80550647, 0.78286994]
    ]
    ,
    [
    [6, 27, 8, 2, 2, 12, 20, 8, 9],
    [0.77064997, 0.7913836, 0.79439026, 0.80753875, 0.78662074]
    ]
    ,
    [
    [2, 20, 13, 2, 3, 10, 16, 14, 14],
    [0.7741262, 0.7872211, 0.79625636, 0.803446, 0.78687155]
    ]
    ,
    [
    [1, 12, 25, 2, 6, 23, 27, 12, 23],
    [0.77446145, 0.7897302, 0.7946332, 0.8069643, 0.78621966]
    ]
    ,
    [
    [4, 10, 30, 3, 5, 15, 16, 27, 30],
    [0.77167046, 0.79226553, 0.79314804, 0.80333865, 0.78594726]
    ]
    ,
    [
    [1, 19, 21, 2, 3, 10, 17, 8, 11],
    [0.77170557, 0.79218316, 0.79457986, 0.80811507, 0.7864752]
    ]
    ,
    [
    [4, 11, 22, 2, 1, 26, 25, 12, 15],
    [0.7713306, 0.78836894, 0.79455405, 0.8016925, 0.782868]
    ]
]

exp_opt_both = [
    [
    [6, 9, 13, 2, 3, 10, 7, 21, 13],
    [0.76400876, 0.7829987, 0.78128344, 0.79019964, 0.7744007]
    ]
    ,
    [
    [5, 16, 26, 5, 1, 24, 22, 15, 12],
    [0.77282953, 0.79303044, 0.7937112, 0.8037511, 0.78582984]
    ]
    ,
    [
    [2, 26, 23, 2, 3, 8, 25, 12, 20],
    [0.76763093, 0.78556395, 0.78696084, 0.7932113, 0.7810889]
    ]
    ,
    [
    [3, 10, 23, 6, 5, 22, 21, 14, 30],
    [0.7749232, 0.7887659, 0.79821175, 0.804335, 0.78557336]
    ]
    ,
    [
    [6, 24, 17, 2, 5, 10, 15, 12, 9],
    [0.7711404, 0.789249, 0.7915764, 0.80103326, 0.7843109]
    ]
    ,
    [
    [3, 10, 11, 1, 2, 8, 20, 25, 20],
    [0.7701651, 0.79308194, 0.7950528, 0.8002121, 0.7865259]
    ]
    ,
    [
    [3, 10, 26, 3, 3, 17, 22, 19, 9],
    [0.7726384, 0.7908612, 0.79450625, 0.80429167, 0.78574246]
    ]
    ,
    [
    [2, 10, 29, 1, 4, 27, 30, 25, 29],
    [0.7728083, 0.79233485, 0.7977434, 0.80158323, 0.78732437]
    ]
    ,
    [
    [6, 10, 12, 1, 3, 17, 25, 25, 12],
    [0.7687207, 0.79085, 0.7855051, 0.796788, 0.78322685]
    ]
    ,
    [
    [1, 22, 12, 1, 3, 22, 8, 23, 26],
    [0.7736404, 0.79395264, 0.8015584, 0.8057956, 0.7873375]
    ]
]

# 5 Folds CV

In [ ]:
for i in range(10):
    for j in range(10):
        func_l, folds5_l = exp_fix_left[i]
        func_r, folds5_r = exp_fix_right[j]
        StatsHelper.plot_box(folds1=folds5_l, name1='fix_left', folds2=folds5_r, name2='fix_right')
    print('--------------------------------------')
        